<a href="https://colab.research.google.com/github/2003UJAN/Fraud-Detection-for-Digital-Payments/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras

In [39]:
df = pd.read_csv("/content/advanced_fraud_transactions.csv")

In [40]:
X = df.drop(columns=["fraudulent"])
y = df["fraudulent"]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [42]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.04, random_state=42)
iso_forest.fit(X_train_scaled)

IsolationForest(contamination=0.04, random_state=42)

In [44]:
y_pred_if = iso_forest.predict(X_test_scaled)
y_pred_if = np.where(y_pred_if == -1, 1, 0)

In [45]:
print("🔹 Isolation Forest Accuracy:", accuracy_score(y_test, y_pred_if))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred_if))

🔹 Isolation Forest Accuracy: 0.8045
🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.97      0.89      3236
           1       0.45      0.10      0.17       764

    accuracy                           0.80      4000
   macro avg       0.63      0.54      0.53      4000
weighted avg       0.75      0.80      0.75      4000



In [46]:
model = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [48]:
model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7564 - loss: 0.5368 - val_accuracy: 0.8085 - val_loss: 0.4308
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8097 - loss: 0.4275 - val_accuracy: 0.8070 - val_loss: 0.4154
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8047 - loss: 0.4146 - val_accuracy: 0.8073 - val_loss: 0.4093
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8072 - loss: 0.3991 - val_accuracy: 0.8058 - val_loss: 0.4056
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8078 - loss: 0.4027 - val_accuracy: 0.7983 - val_loss: 0.4056
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8098 - loss: 0.3999 - val_accuracy: 0.8070 - val_loss: 0.4018
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8145 - loss: 0.3893 - val_accuracy: 0.8043 - val_loss: 0.4001
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8065 - loss: 0.3949 - val_accuracy: 0.

In [49]:
y_pred_nn = (model.predict(X_test_scaled) > 0.5).astype("int32")

print("🔹 Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred_nn))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
🔹 Neural Network Accuracy: 0.80475
🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.99      0.89      3236
           1       0.39      0.04      0.07       764

    accuracy                           0.80      4000
   macro avg       0.60      0.51      0.48      4000
weighted avg       0.73      0.80      0.73      4000



In [50]:
model.save("fraud_detection_model.h5")